## Redes Neurais Artificiais 2020.1

**Disciplina**: Redes Neurais Artificiais 2020.1  
**Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
**Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta terceira parte do Projeto Prático 3 diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Testando Redes Neurais sem os Atributos Categórios

1. Abra a base de dados em questão
2. Elimine todas as colunas relativas aos atributos categóricos
3. Armazene o atributo alvo em uma variável y e os atributos preditores em uma variável X
4. Efetue uma partição holdout 70/30 com o sklearn, distribuindo os exemplos de maneira aleatória
5. Efetue o escalonamento dos atributos

### Escalonando os atributos

O treinamento de uma rede neural artificial é mais eficiente quando os valores que lhes são fornecidos como entrada são pequenos, pois isto favorece a convergência. Isto é feito escalonando-se todos os atributos para o intervalo [0,1], mas precisa ser feito de maneira cautelosa, para que informações do conjunto de teste não sejam fornecidas no treinamento.

Há duas estratégias para tal escalonamento: normalização e padronização. Ambas possuem características particulares, vantagens e limitações, como é possível ver aqui: https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/


No nosso caso, vamos usar a padronização. Assim, com os atributos preditores do treinamento, isto é, X_train, deve-se subtrair a média e dividir pelo desvio padrão:

X_train_std = (X_train - np.mean(X_train))/np.std(X_train)

Em seguida, o mesmo deve ser feito com os atributos preditores do conjunto de testes, mas com padronização relativa ao conjunto de treinamento:

X_test_std = (X_test - np.mean(X_train))/np.std(X_train)

Se todo o conjunto X for utilizado na padronização, a rede neural receberá informações do conjunto de teste por meio da média e variância utilizada para preparar os dados de treinamento, o que não é desejável.


### Continuando

5. Treine uma rede neural multilayer perceptron para este problema com uma única camada e dez neurônios  
    5.1 Utilize a função de ativação ReLU  
    5.2 Utilize o solver Adam    
    5.3 Imprima o passo a passo do treinamento    
    5.4 Utilize o número máximo de épocas igual a 300  
6. Com o modelo em questão, após o treinamento, apresente:  
    6.1 Matriz de confusão para o conjunto de teste  
    6.2 Acurácia  
    6.3 F-Score  
    6.4 Precisão  
    6.5 Revocação  
7. Repita o treinamento da mesma rede anterior sem imprimir o passo a passo (verbose False) por 100 vezes  
    7.1 Cada uma destas repetições deve ser feita com uma nova partição Holdout  
    7.2 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  
8. Repita por 100 vezes o treinamento desta mesma rede, mas utilizando o otimizador SGD  
    8.1 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  
9. Houve influência da escolha do otimizador no desempenho da rede?

In [ ]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import random
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import plot_confusion_matrix
from joblib import Parallel, delayed

In [ ]:
# Montando o drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Extraindo o data frame
df = pd.read_csv('/content/drive/Shared drives/RNA/covtype.csv', sep=',')

In [ ]:
# Limpando atributos categóricos

categoricos = []

for i in range(40):
  if i < 4:
    categoricos.append("Wilderness_Area"+str(i+1))
  categoricos.append("Soil_Type"+str(i+1))

df = df.drop(columns=categoricos)  

# Separação dos atributos alvo e preditores.
X_atributo_preditor = df.drop(columns=["Cover_Type"])
Y_atributo_alvo = df["Cover_Type"]

# Separação dos dados para treino e teste. Proporção: 70% para treino e 30% para teste
X_train, X_test, Y_train, Y_test = train_test_split(X_atributo_preditor, Y_atributo_alvo, test_size=0.3, train_size=0.7)

# Padronozação
X_train_std = (X_train - np.mean(X_train))/np.std(X_train)  
X_test_std = (X_test - np.mean(X_train))/np.std(X_train)

## Discussão

Nos passos anteriores, você avaliou o desempenho de uma única rede neural que contém os seguintes parâmetros: uma única camada oculta com 10 neurônios e função de ativação ReLU. O otimizador utilizado, quer seja SGD ou ADAM, trata-se do algoritmo para aproximar o gradiente do erro. Neste sentido, a escolha do otimizador é um hiperparâmetro, pois diz respeito a como a rede neural definida previamente atuará "em tempo de execução"  durante o processo de treinamento. Também são hiperparâmetros a quantidade de épocas, a taxa de aprendizado inicial, dentre outros.

Cabe alientar também que você efetuou o treinamento desta rede por 100 vezes e apresentou os resultados em termos de média +- desvio padrão. Lembre-se que em uma rede neural há a inicialização aleatória de pesos e, em consequência, o desempenho delas está sujeito à uma flutuação estocástica. A execução destas múltiplas vezes faz com que eliminemos algum viés introduzido por uma boa ou má "sorte" na escolha de pesos no caso de uma única execução.

Você também aprendeu uma estratégia para escalonar os atributos para uma melhor convergência da rede. Utilize-a em todos os treinamentos e testes propostos a seguir.

## Propondo Novas Arquiteturas

Variando  os parâmetros (uma ou duas camadas ocultas, com diferente números de neurônios em cada uma delas e a função de ativação) e o hiperparâmetros solver (Adam ou SGD) e o número de épocas (100,150 e 200), atenda ao que se pede:

1. Proponha 10 arquiteturas distintas de RNAs para o problema em questão, à sua escolha
2. Avalie cada uma das arquiteturas perante todos os hiperparâmetros apresentados por 100 vezes
3. Como resultado da avaliação, apresente:  
    3.1 Top-3 melhores redes no tocante à F-Score e Acurácia  
    3.2 Repetição em que houve o melhor desempenho de cada uma dessas redes: ilustre tp, tf, fp e fn  

## Estimando o número de neurônios

Um dos problemas de pesquisa com redes neurais artificiais consiste na determinação do número de neurônios em sua arquitetura. Embora não seja possível definir a priori qual rede neural é adequada para um problema, pois isto só é possível mediante uma busca exaustiva, há regras na literatura que sugerem o número de neurônios escondidos, tal como a regra da Pirâmide Geométrica, dada a seguir:

$$N_h = \alpha \cdot \sqrt{N_i \cdot N_o},$$

em que $N_h$ é o número de neurônios ocultos (a serem distribuídos em uma ou duas camadas ocultas), $N_i$ é o número de neurônios na camada de entrada e $N_o$ é o número de neurônios na camada de saída. 

1. Consulte a documentação da classe MLPClassifier (disponível em https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) e obtenha os valores de $N_i$ e $N_h$.
2. Teste os valores de $\alpha$ como sendo iguais a $0.5$, $2$ e $3$.
3. Proponha pelo menos 30 redes neurais segundo a regra da pirâmide geométrica e teste-as nos mesmos termos estabelecidos anterioremente  (solver, épocas, etc.)  
    3.1 Apresente as top-3 melhores redes no tocante à F-Score e Acurácia  

## 1. 
Ni é a quantidade de atributos preditores, ou seja, 10. No é a quantidade de classes de resposta, ou seja, 7.

O Nh é a quantidade de neurônios escondidos e depende de um alpha, sendo assim, $$N_h = \alpha \cdot \sqrt{70},$$

In [ ]:
def train_model(params, id, amount):
  v_acuracia = []
  v_f1score = []
  models = []
  for i in range(amount):
    print(i, end=" ")

    # Separação dos dados para treino e teste. Proporção: 70% para treino e 30% para teste
    X_train, X_test, Y_train, Y_test = train_test_split(X_atributo_preditor, Y_atributo_alvo, test_size=0.3, train_size=0.7)

    # Padronozação
    X_train_std = (X_train - np.mean(X_train))/np.std(X_train)  
    X_test_std = (X_test - np.mean(X_train))/np.std(X_train)
    
    clf = MLPClassifier(hidden_layer_sizes=params['hl'],
                        max_iter=params['max_iter'],
                        solver=params['solver'],
                        activation=params['activation'],
                        random_state=1,  
                        verbose=False
                        ).fit(X_train_std, Y_train)
    models.append(clf)
    # Extraindo a predi4/1AfDhmrjR-kEkP3QXaB0On5IMjX1MvUFQCJDqAdd6YyDmZZrbrvYn7VadlJUção
    predicao = clf.predict(X_test_std)

    v_acuracia.append(accuracy_score(Y_test, predicao))
    v_f1score.append(f1_score(Y_test, predicao, average="macro"))

  print("\n\nResultados obtidos usando o solver " + str(id) + ":\n")

  print("Média da Acurácia:", round(np.mean(v_acuracia), 4))
  print("Desvio Padrão da Acurácia:", round(np.std(v_acuracia), 4))
  print("Média do F-Score:", round(np.mean(v_f1score), 4))
  print("Desvio Padrão do F-Score:", round(np.std(v_f1score), 4))

  idx = v_acuracia.index(max(v_acuracia))
  matrix = confusion_matrix(Y_test, models[idx].predict(X_test_std))

  fp = matrix.sum(axis=0) - np.diag(matrix)
  fn = matrix.sum(axis=1) - np.diag(matrix)
  tp = np.diag(matrix)
  tn = matrix.sum() - (fp + fn + tp)
  return {
      'accuracy': round(np.mean(v_acuracia), 4),
      'f1-score': round(np.mean(v_f1score), 4),
      'id': id
  }, {
      'fp': fp,
      'fn': fn,
      'tp': tp,
      'tn': tn
  }


In [ ]:
params = {
            'hl': (10, 10),
            'max_iter': 100,
            'solver': 'adam',
            'activation': 'identity',
            'layers': 2,
            'neuronios': 5
         }

ni = 10
no = 7

alphas = [0.5, 2, 3]

funcaoAtivacao = ["identity", "logistic", "tanh", "relu"]
slv = ["adam", "sgd"]
epocas = [5, 7, 10] #[100, 150, 200]

def getNh(alpha):
  return alpha * sqrt(ni * no)

nh = []

for i in alphas:
  nh.append(int(getNh(i)))

arquiteturas = []
resultados = []
nh

[4, 16, 25]

In [ ]:

params = [
         {
            'hl':(4),
            'max_iter': 50,
            'solver': 'adam',
            'activation': 'identity'
         },
         {
            'hl':(10, 6),
            'max_iter': 75,
            'solver': 'adam',
            'activation': 'identity'
         },
         {
            'hl':(12, 13),
            'max_iter': 75,
            'solver': 'adam',
            'activation': 'relu'
         },
         {
            'hl':(10, 5),
            'max_iter': 100,
            'solver': 'adam',
            'activation': 'tanh'
         },
         {
            'hl':(15, 10),
            'max_iter': 50,
            'solver': 'sgd',
            'activation': 'logistic'
         },
         {
            'hl':(15, 10),
            'max_iter': 50,
            'solver': 'adam',
            'activation': 'tanh'
         }          
]
params

[{'activation': 'identity', 'hl': 4, 'max_iter': 50, 'solver': 'adam'},
 {'activation': 'identity', 'hl': (10, 6), 'max_iter': 75, 'solver': 'adam'},
 {'activation': 'relu', 'hl': (12, 13), 'max_iter': 75, 'solver': 'adam'},
 {'activation': 'tanh', 'hl': (10, 5), 'max_iter': 100, 'solver': 'adam'},
 {'activation': 'logistic', 'hl': (15, 10), 'max_iter': 50, 'solver': 'sgd'},
 {'activation': 'tanh', 'hl': (15, 10), 'max_iter': 50, 'solver': 'adam'}]

In [ ]:
models_result = []
for id, param in enumerate(params):
  model_result, pr = train_model(params=param, id=id, amount=7)
  models_result.append(model_result)

0 1 2 3 4 5 6 

Resultados obtidos usando o solver 0:

Média da Acurácia: 0.6961
Desvio Padrão da Acurácia: 0.0015
Média do F-Score: 0.4079
Desvio Padrão do F-Score: 0.0068
0 1 2 3 4 5 6 

Resultados obtidos usando o solver 1:

Média da Acurácia: 0.6967
Desvio Padrão da Acurácia: 0.0009
Média do F-Score: 0.4269
Desvio Padrão do F-Score: 0.0082
0 

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (75) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


1 

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (75) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


2 

...

5 

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


6 

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)




Resultados obtidos usando o solver 5:

Média da Acurácia: 0.7457
Desvio Padrão da Acurácia: 0.0021
Média do F-Score: 0.5564
Desvio Padrão do F-Score: 0.0089


In [ ]:
models_result.sort(key=lambda model: model['accuracy'], reverse=True)
print("Best Accuracy")
pd.DataFrame(models_result).head(6)

Best Accuracy


,accuracy,f1-score,id
0,0.7457,0.5564,5
1,0.7387,0.5837,2
2,0.7268,0.5327,3
3,0.6988,0.3469,4
4,0.6967,0.4269,1
5,0.6961,0.4079,0


In [ ]:
models_result.sort(key=lambda model: model['f1-score'], reverse=True)
print("Best f1-score")
pd.DataFrame(models_result).head(6)

Best f1-score


,accuracy,f1-score,id
0,0.7387,0.5837,2
1,0.7457,0.5564,5
2,0.7268,0.5327,3
3,0.6967,0.4269,1
4,0.6961,0.4079,0
5,0.6988,0.3469,4


## Testando as Redes Neurais com Atributos Categóricos

1. Considere as 6 redes neurais obtidas nos dois top-3 anteriores (arquiteturas próprias e regra da pirâmide geométrica)
2. Com todos os atributos preditores da base de dados original, incluindo os categóricos, treine e teste estas mesmas redes por 100 repetições  
    2.1 Considere o melhor otimizador para cada uma delas  
    2.2 Faça uso de 200 épocas para treinamento  
    2.2 Apresente os resultados de acurácia e F-Score em termos da média +- dp para cada arquitetura
3. Apresente o gráfico boxplot para o F-Score das 6 arquiteturas perante as 100 repetições

## Considerações Parciais

1. É possível identificar uma rede com desempenho superior às demais?
2. Qual estratégia mostrou-se mais producente para a obtenção de boas arquiteturas (Estratégia Própria ou Pirâmide Geométrica)? Por quê?
3. Considerar os atributos categóricos trouxe melhorias? Justifique.
4. Um número maior de épocas trouxe melhorias?
5. Qual a maior dificuldade de resolução do problema proposto perante as RNAs?